In [9]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Import training data to pandas dataframe
df = pd.read_csv("train.csv")
df.hist(bins=50, figsize=(22,16))
# Trimming Down
df_bm = df.loc[:,['Neighborhood','GrLivArea','YearBuilt','OverallQual','CentralAir','SalePrice','FullBath','HalfBath','BsmtFullBath','BsmtHalfBath','TotalBsmtSF','BsmtUnfSF']]


In [10]:
# Creating total bathroom count TotBathroom
TotBathroom = df_bm.FullBath + 0.5 * df_bm.HalfBath \
+ df_bm.BsmtFullBath + (0.5) * df_bm.BsmtHalfBath

df_bm['TotBathroom'] = TotBathroom

# Remove the one's we don't need anymore
del df_bm['FullBath']
del df_bm['HalfBath']
del df_bm['BsmtFullBath']
del df_bm['BsmtHalfBath']

In [11]:
# Creating total square foot TotSF
TotSF = df_bm.GrLivArea + df_bm.TotalBsmtSF - df_bm.BsmtUnfSF
# Adding to dataset
df_bm['TotSF'] = TotSF

# Remove the ones we don't need anymore
del df_bm['GrLivArea']
del df_bm['TotalBsmtSF']
del df_bm['BsmtUnfSF']


,Neighborhood,YearBuilt,OverallQual,CentralAir,SalePrice,TotBathroom,TotSF
0,CollgCr,2003,7,Y,208500,3.5,2416
1,Veenker,1976,6,Y,181500,2.5,2240
2,CollgCr,2001,7,Y,223500,3.5,2272
3,Crawfor,1915,7,Y,140000,2.0,1933
4,NoRidge,2000,8,Y,250000,3.5,2853


In [13]:
# Data Preprocessing

# deal with missing values in textual columns 
#replace missing neighborhood with a dummy variable 
df_bm['Neighborhood'].fillna('fill')
#Replace missing ac classification with Y since more likely
#and allows us to maintain binary classification
df_bm['CentralAir'].fillna('Y')

df_bm[:5]

,Neighborhood,YearBuilt,OverallQual,CentralAir,SalePrice,TotBathroom,TotSF
0,CollgCr,2003,7,Y,208500,3.5,2416
1,Veenker,1976,6,Y,181500,2.5,2240
2,CollgCr,2001,7,Y,223500,3.5,2272
3,Crawfor,1915,7,Y,140000,2.0,1933
4,NoRidge,2000,8,Y,250000,3.5,2853
